In [2]:
import pandas as pd 
import numpy as np 
from keras.utils.np_utils import to_categorical   

import recordlinker

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore', 'info')

Using TensorFlow backend.


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
iowa_matches = pd.read_csv('/Users/kailinlu/Desktop/QMSSWork/RecordLinking/recordlinker/recordlinker/data/iowa_matches.csv')
iowa_nonmatches = pd.read_csv('/Users/kailinlu/Desktop/QMSSWork/RecordLinking/recordlinker/recordlinker/data/iowa_nonmatches.csv')

In [5]:
iowa_matches.head()

,uid1915,fname1915,lname1915,fullname1915,yob1915,hhid,fname1940,lname1940,fullname1940,yob1940,uid-hhid
0,uid0910071227,donald d,cutler,donald d cutler,1911,19067,donald dean,cutler,donald dean cutler,1911,uid0910071227-19067
1,uid0063131339,homer,taylor,homer taylor,1912,71505,homer ellis,taylor,homer ellis taylor,1912,uid0063131339-71505
2,uid0044088276,earl,stearnes,earl stearnes,1899,109708,earl,stearns,earl stearns,1900,uid0044088276-109708
3,uid0067053130,theodore,hornaday,theodore hornaday,1904,108304,theodore i,harnaday,theodore i harnaday,1904,uid0067053130-108304
4,uid0066046148,jack r,turner,jack r turner,1907,105092,jack r,turner,jack r turner,1907,uid0066046148-105092


In [6]:
union_matches = pd.read_csv('/Users/kailinlu/Desktop/QMSSWork/RecordLinking/recordlinker/recordlinker/data/unionarmy_matches.csv')

In [7]:
union_matches.head()

,recidnum,recname1,recname2,last1,first1,last2,first2
0,100501001,anson charles h,anson charles h,anson,charles h,anson,charles h
1,100501002,allsheskey theodore f,allsheskey theodore f,allsheskey,theodore f,allsheskey,theodore f
2,100501003,bill charles w,bill c w,bill,charles w,bill,c w
3,100501004,bradley george a,bradley george a,bradley,george a,bradley,george a
4,100501005,bunitt william n,burritt william n,bunitt,william n,burritt,william n


### String Embedding Example

In [42]:
from recordlinker.preprocess import embed_letters, embed_shingles, disembed_letters, disembed_shingles

name = 'kailin lu'
max_length = 12 

print('Embed Letters: \n', 
      embed_letters(name, max_length), 
      disembed_letters(embed_letters(name, max_length))) 

print('Embed Letters Normalized: \n', 
      embed_letters(name, max_length, normalize=True), 
      disembed_letters(embed_letters(name, max_length, normalize=True)))

print('Embed 2-Shingles: \n', 
      embed_shingles(name, max_length), 
      disembed_shingles(embed_shingles(name, max_length))) 

print('Embed 2-Shingles Normalized: \n', 
      embed_shingles(name, max_length, normalize=True), 
      disembed_shingles(embed_shingles(name, max_length, normalize=True)))

Embed Letters: 
 [11  1  9 12  9 14 27 12 21  0  0  0] kailin lu
Embed Letters Normalized: 
 [0.40740741 0.03703704 0.33333333 0.44444444 0.33333333 0.51851852
 1.         0.44444444 0.77777778 0.         0.         0.        ] kailin lu
Embed 2-Shingles: 
 [261   8 219 295 221 364 688 306   0   0   0   0] kailin lu
Embed 2-Shingles Normalized: 
 [0.35753425 0.0109589  0.3        0.40410959 0.30273973 0.49863014
 0.94246575 0.41917808 0.         0.         0.         0.        ] kailin lu


## Train and save autoencoders

### Dense

In [71]:
ORIG_LENGTH = 12
BATCH_SIZE = 32
ENCODE_DIM = [256, 128] 
DECODE_DIM = [128, 256]
LR = 1e-4
EPOCHS=301
LATENT_DIM = [16,24]

# Embed letters 
namesA = recordlinker.preprocess.create_training_set(iowa_matches, 'lname1915',
                                                     max_length=ORIG_LENGTH, 
                                                     embed_type='letters', 
                                                     normalize=True)
namesB = recordlinker.preprocess.create_training_set(iowa_matches, 'lname1940',
                                                     max_length=ORIG_LENGTH, 
                                                     embed_type='letters', 
                                                     normalize=True)

for latent_dim in LATENT_DIM: 
    save_path = '/Users/kailinlu/Desktop/QMSSWork/RecordLinking/models/dense_letter_{}/'.format(latent_dim)
    run_id = 'dense_{}'.format(latent_dim)
    vae = recordlinker.model.VAE(batch_size=BATCH_SIZE,
                                 orig_dim=ORIG_LENGTH, 
                                 latent_dim=latent_dim,
                                 encode_dim=ENCODE_DIM,
                                 decode_dim=DECODE_DIM,
                                 lr=LR)

    model, encoder, decoder = vae.train(namesA, namesB, 
                                        epochs=EPOCHS, 
                                        run_id=run_id,
                                        save_path=save_path,
                                        optimizer='adam', 
                                        tensorboard=True, 
                                        earlystop=True,
                                        earlystop_patience=6,
                                        reconstruct=True, 
                                        reconstruct_display=10)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12)           0                                            
__________________________________________________________________________________________________
enc_0 (Dense)                   (None, 256)          3328        input_1[0][0]                    
__________________________________________________________________________________________________
enc_1 (Dense)                   (None, 128)          32896       enc_0[0][0]                      
__________________________________________________________________________________________________
mu (Dense)                      (None, 16)           2064        enc_1[0][0]                      
__________________________________________________________________________________________________
log_sigma 

3456/3456 [==============================] - 0s 89us/step - loss: 0.6694 - acc: 0.6476 - val_loss: 0.7228 - val_acc: 0.6458
Epoch 37/301
3456/3456 [==============================] - 0s 80us/step - loss: 0.6625 - acc: 0.6548 - val_loss: 0.7265 - val_acc: 0.6667
Epoch 38/301
3456/3456 [==============================] - 0s 84us/step - loss: 0.6618 - acc: 0.6513 - val_loss: 0.7169 - val_acc: 0.6458
Epoch 39/301
3456/3456 [==============================] - 0s 88us/step - loss: 0.6568 - acc: 0.6496 - val_loss: 0.7285 - val_acc: 0.6238
Epoch 40/301
3456/3456 [==============================] - 0s 78us/step - loss: 0.6592 - acc: 0.6502 - val_loss: 0.7139 - val_acc: 0.6678
Epoch 41/301
3456/3456 [==============================] - 0s 80us/step - loss: 0.6494 - acc: 0.6698 - val_loss: 0.7299 - val_acc: 0.6065
Sample Reconstructions:
{'Orig': 'patrick', 'Pred:': 'hcsnkgb     '}
{'Orig': 'larson', 'Pred:': 'kcupmoa     '}
{'Orig': 'dolan', 'Pred:': 'hnpciaba    '}
{'Orig': 'dannenfeldt', 'Pred:': 'd

Train on 3456 samples, validate on 864 samples
Epoch 1/301
3456/3456 [==============================] - 0s 132us/step - loss: 3.6610 - acc: 0.2729 - val_loss: 1.9442 - val_acc: 0.4294
Sample Reconstructions:
{'Orig': 'jacobs', 'Pred:': 'ifmmeneba   '}
{'Orig': 'casey', 'Pred:': 'jjrgngca    '}
{'Orig': 'haben', 'Pred:': 'kmljjfdba   '}
{'Orig': 'mehaffey', 'Pred:': 'kkllkkjgeccc'}
{'Orig': 'lacy', 'Pred:': 'jiincfca    '}
Epoch 2/301
3456/3456 [==============================] - 0s 78us/step - loss: 1.4818 - acc: 0.5014 - val_loss: 1.2656 - val_acc: 0.5428
Epoch 3/301
3456/3456 [==============================] - 0s 87us/step - loss: 1.0978 - acc: 0.5720 - val_loss: 1.0533 - val_acc: 0.6111
Epoch 4/301
3456/3456 [==============================] - 0s 88us/step - loss: 0.9510 - acc: 0.5952 - val_loss: 0.9633 - val_acc: 0.5914
Epoch 5/301
3456/3456 [==============================] - 0s 93us/step - loss: 0.8762 - acc: 0.6050 - val_loss: 0.9197 - val_acc: 0.6204
Epoch 6/301
3456/3456 [=======

In [ ]:
ORIG_LENGTH = 12
BATCH_SIZE = 32
ENCODE_DIM = [256, 128] 
DECODE_DIM = [128, 256]
LR = 1e-4
EPOCHS=100
LATENT_DIM = [4,8,16,24]

# Embed letters 
namesA = recordlinker.preprocess.create_training_set(union_matches, 'first1',
                                                     max_length=ORIG_LENGTH, 
                                                     embed_type='letters', 
                                                     normalize=False)
namesB = recordlinker.preprocess.create_training_set(union_matches, 'first2',
                                                     max_length=ORIG_LENGTH, 
                                                     embed_type='letters', 
                                                     normalize=False)

namesA = namesA[:-34,:]
namesB = namesB[:-34,:]

classes = 28
namesA_onehot = to_categorical(namesA, classes) 
namesB_onehot = to_categorical(namesB, classes)

for latent_dim in LATENT_DIM: 
    save_path = '/Users/kailinlu/Desktop/QMSSWork/RecordLinking/models/lstm_letter_{}_union_first/'.format(latent_dim)
    run_id = 'lstm_{}'.format(latent_dim)
    lstm_vae = recordlinker.model.LSTMVAE(batch_size=BATCH_SIZE, 
                                          timesteps=ORIG_LENGTH, 
                                          orig_dim=classes,
                                          latent_dim=latent_dim,
                                          encode_dim=64, 
                                          decode_dim=64,
                                          lr=LR) 
    model_lstm, model_encoder, model_decoder = lstm_vae.train(namesA_onehot, namesB_onehot, 
                                                              epochs=EPOCHS, 
                                                              run_id=run_id, 
                                                              save_path=save_path, 
                                                              validation_split=0,
                                                              tensorboard=False,
                                                              reconstruct=True, 
                                                              reconstruct_display=10)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12, 28)       0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 12, 64)       23808       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 12, 64)       33024       lstm_1[0][0]                     
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 768)          0           lstm_2[0][0]                     
__________________________________________________________________________________________________
mu (Dense)

/usr/local/lib/python3.6/site-packages/Keras-2.1.6-py3.6.egg/keras/callbacks.py:534: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc


Sample Reconstructions:
{'Orig': 'noah w', 'Pred:': 'james'}
{'Orig': 'henry', 'Pred:': 'joory'}
{'Orig': 'james f', 'Pred:': 'jamee'}
{'Orig': 'richard', 'Pred:': 'jaliaa '}
{'Orig': 'nathaniel', 'Pred:': 'jallaai  '}
Epoch 2/100
44064/44064 [==============================] - 41s 933us/step - loss: 15.5298 - acc: 0.6722
Epoch 3/100
44064/44064 [==============================] - 41s 924us/step - loss: 13.2898 - acc: 0.7241
Epoch 4/100
44064/44064 [==============================] - 40s 913us/step - loss: 11.9687 - acc: 0.7574
Epoch 5/100
44064/44064 [==============================] - 39s 894us/step - loss: 11.2245 - acc: 0.7758
Epoch 6/100
44064/44064 [==============================] - 40s 908us/step - loss: 10.7193 - acc: 0.7865
Epoch 7/100
44064/44064 [==============================] - 41s 923us/step - loss: 10.3212 - acc: 0.7984
Epoch 8/100
44064/44064 [==============================] - 40s 896us/step - loss: 10.0205 - acc: 0.8063
Epoch 9/100
44064/44064 [============================

44064/44064 [==============================] - 41s 941us/step - loss: 7.2707 - acc: 0.8661
Epoch 67/100
44064/44064 [==============================] - 41s 939us/step - loss: 7.2667 - acc: 0.8661
Epoch 68/100
44064/44064 [==============================] - 41s 920us/step - loss: 7.2433 - acc: 0.8667
Epoch 69/100
44064/44064 [==============================] - 40s 914us/step - loss: 7.2285 - acc: 0.8672
Epoch 70/100
44064/44064 [==============================] - 42s 956us/step - loss: 7.2226 - acc: 0.8671
Epoch 71/100
44064/44064 [==============================] - 41s 931us/step - loss: 7.1915 - acc: 0.8679
Sample Reconstructions:
{'Orig': 'jesse', 'Pred:': 'jesse'}
{'Orig': 'thomas', 'Pred:': 'thomas'}
{'Orig': 'john', 'Pred:': 'john'}
{'Orig': 'george r', 'Pred:': 'george r'}
{'Orig': 'david', 'Pred:': 'david'}
Epoch 72/100
44064/44064 [==============================] - 40s 912us/step - loss: 7.1897 - acc: 0.8679
Epoch 73/100
44064/44064 [==============================] - 41s 922us/step 

44064/44064 [==============================] - 41s 941us/step - loss: 10.5453 - acc: 0.7898
Epoch 4/100
44064/44064 [==============================] - 41s 931us/step - loss: 9.4257 - acc: 0.8218
Epoch 5/100
44064/44064 [==============================] - 42s 957us/step - loss: 8.7068 - acc: 0.8402
Epoch 6/100
44064/44064 [==============================] - 41s 934us/step - loss: 8.2470 - acc: 0.8513
Epoch 7/100
44064/44064 [==============================] - 42s 956us/step - loss: 7.9318 - acc: 0.8583
Epoch 8/100
44064/44064 [==============================] - 40s 919us/step - loss: 7.6863 - acc: 0.8634
Epoch 9/100
44064/44064 [==============================] - 41s 923us/step - loss: 7.5043 - acc: 0.8671
Epoch 10/100
44064/44064 [==============================] - 41s 926us/step - loss: 7.3635 - acc: 0.8702
Epoch 11/100
44064/44064 [==============================] - 41s 924us/step - loss: 7.2180 - acc: 0.8730
Sample Reconstructions:
{'Orig': 'samuel j', 'Pred:': 'samuel b'}
{'Orig': 'john',

44064/44064 [==============================] - 42s 949us/step - loss: 5.4550 - acc: 0.9079
Epoch 70/100
44064/44064 [==============================] - 41s 930us/step - loss: 5.4344 - acc: 0.9084
Epoch 71/100
44064/44064 [==============================] - 40s 917us/step - loss: 5.4127 - acc: 0.9088
Sample Reconstructions:
{'Orig': 'frank e', 'Pred:': 'frank f'}
{'Orig': 'patrick', 'Pred:': 'patrick'}
{'Orig': 'samuel', 'Pred:': 'samuel'}
{'Orig': 'james', 'Pred:': 'james'}
{'Orig': 'phelps o', 'Pred:': 'pheles e'}
Epoch 72/100
44064/44064 [==============================] - 41s 931us/step - loss: 5.4182 - acc: 0.9086
Epoch 73/100
44064/44064 [==============================] - 42s 942us/step - loss: 5.3971 - acc: 0.9088
Epoch 74/100
44064/44064 [==============================] - 42s 948us/step - loss: 5.4037 - acc: 0.9085
Epoch 75/100
44064/44064 [==============================] - 40s 918us/step - loss: 5.3885 - acc: 0.9091
Epoch 76/100
44064/44064 [==============================] - 41s 9

Epoch 6/100
44064/44064 [==============================] - 42s 957us/step - loss: 7.3818 - acc: 0.8730
Epoch 7/100
44064/44064 [==============================] - 42s 951us/step - loss: 7.0978 - acc: 0.8792
Epoch 8/100
44064/44064 [==============================] - 41s 935us/step - loss: 6.9005 - acc: 0.8836
Epoch 9/100
44064/44064 [==============================] - 41s 935us/step - loss: 6.7291 - acc: 0.8878
Epoch 10/100
44064/44064 [==============================] - 41s 936us/step - loss: 6.6030 - acc: 0.8906
Epoch 11/100
44064/44064 [==============================] - 42s 951us/step - loss: 6.4718 - acc: 0.8938
Sample Reconstructions:
{'Orig': 'peter d', 'Pred:': 'peter c'}
{'Orig': 'alexander e', 'Pred:': 'alexander '}
{'Orig': 'alexander', 'Pred:': 'alexander'}
{'Orig': 'mark l', 'Pred:': 'mark d'}
{'Orig': 'abraham', 'Pred:': 'abraham'}
Epoch 12/100
44064/44064 [==============================] - 42s 945us/step - loss: 6.3816 - acc: 0.8957
Epoch 13/100
44064/44064 [=================

44064/44064 [==============================] - 42s 952us/step - loss: 5.0733 - acc: 0.9160
Epoch 73/100
44064/44064 [==============================] - 45s 1ms/step - loss: 5.0474 - acc: 0.9166
Epoch 74/100
44064/44064 [==============================] - 45s 1ms/step - loss: 5.0499 - acc: 0.9165
Epoch 75/100
44064/44064 [==============================] - 42s 949us/step - loss: 5.0364 - acc: 0.9166
Epoch 76/100
44064/44064 [==============================] - 44s 998us/step - loss: 5.0187 - acc: 0.9172
Epoch 77/100
44064/44064 [==============================] - 45s 1ms/step - loss: 5.0268 - acc: 0.9166
Epoch 78/100
44064/44064 [==============================] - 43s 971us/step - loss: 5.0013 - acc: 0.9173
Epoch 79/100
44064/44064 [==============================] - 43s 979us/step - loss: 5.0019 - acc: 0.9171
Epoch 80/100
44064/44064 [==============================] - 45s 1ms/step - loss: 4.9916 - acc: 0.9172ETA
Epoch 81/100
44064/44064 [==============================] - 43s 969us/step - loss:

44064/44064 [==============================] - 43s 983us/step - loss: 6.3026 - acc: 0.8980
Sample Reconstructions:
{'Orig': 'hugh', 'Pred:': 'hugh'}
{'Orig': 'cornelius w', 'Pred:': 'cornelius w'}
{'Orig': 'thomas', 'Pred:': 'thomas'}
{'Orig': 'john m', 'Pred:': 'john m'}
{'Orig': 'rudolph', 'Pred:': 'rodolph'}
Epoch 12/100
44064/44064 [==============================] - 45s 1ms/step - loss: 6.2148 - acc: 0.8993
Epoch 13/100
44064/44064 [==============================] - 46s 1ms/step - loss: 6.1301 - acc: 0.9010
Epoch 14/100
44064/44064 [==============================] - 45s 1ms/step - loss: 6.0674 - acc: 0.9019
Epoch 15/100
44064/44064 [==============================] - 44s 1ms/step - loss: 6.0042 - acc: 0.9031
Epoch 16/100
44064/44064 [==============================] - 45s 1ms/step - loss: 5.9422 - acc: 0.9042
Epoch 17/100
44064/44064 [==============================] - 43s 979us/step - loss: 5.9137 - acc: 0.9046
Epoch 18/100
44064/44064 [==============================] - 43s 979us/ste

In [72]:
# Embed shingles
ORIG_LENGTH = 12
BATCH_SIZE = 32
ENCODE_DIM = [256, 128] 
DECODE_DIM = [128, 256]
LR = 1e-4
EPOCHS=301
EMBED_TYPE = 'shingles'
LATENT_DIM = [16,24]

# Embed letters 
namesA = recordlinker.preprocess.create_training_set(iowa_matches, 'lname1915',
                                                     max_length=ORIG_LENGTH, 
                                                     embed_type=EMBED_TYPE, 
                                                     normalize=True)
namesB = recordlinker.preprocess.create_training_set(iowa_matches, 'lname1940',
                                                     max_length=ORIG_LENGTH, 
                                                     embed_type=EMBED_TYPE, 
                                                     normalize=True)

for latent_dim in LATENT_DIM: 
    save_path = '/Users/kailinlu/Desktop/QMSSWork/RecordLinking/models/dense_shingle_{}_iowa_last/'.format(latent_dim)
    run_id = 'dense_{}'.format(latent_dim)
    vae = recordlinker.model.VAE(batch_size=BATCH_SIZE,
                                 orig_dim=ORIG_LENGTH, 
                                 latent_dim=latent_dim,
                                 encode_dim=ENCODE_DIM,
                                 decode_dim=DECODE_DIM,
                                 lr=LR)

    model, encoder, decoder = vae.train(namesA, namesB, 
                                        epochs=EPOCHS, 
                                        run_id=run_id,
                                        save_path=save_path,
                                        optimizer='adam', 
                                        tensorboard=True, 
                                        earlystop=True,
                                        earlystop_patience=6,
                                        reconstruct=True, 
                                        reconstruct_type='s',
                                        reconstruct_display=10)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12)           0                                            
__________________________________________________________________________________________________
enc_0 (Dense)                   (None, 256)          3328        input_1[0][0]                    
__________________________________________________________________________________________________
enc_1 (Dense)                   (None, 128)          32896       enc_0[0][0]                      
__________________________________________________________________________________________________
mu (Dense)                      (None, 16)           2064        enc_1[0][0]                      
__________________________________________________________________________________________________
log_sigma 

Train on 3456 samples, validate on 864 samples
Epoch 1/301
3456/3456 [==============================] - 0s 132us/step - loss: 3.6671 - acc: 0.3906 - val_loss: 1.8282 - val_acc: 0.6169
Sample Reconstructions:
{'Orig': 'byard', 'Pred:': 'ktgnfcaaaaaae'}
{'Orig': 'mcdonnell', 'Pred:': 'lknomkiffeeeu'}
{'Orig': 'solnar', 'Pred:': 'knkmebaaaaaac'}
{'Orig': 'moore', 'Pred:': 'jvhkfcaaaaaac'}
{'Orig': 'kuper', 'Pred:': 'jsnfecaaaaaac'}
Epoch 2/301
3456/3456 [==============================] - 0s 78us/step - loss: 1.3385 - acc: 0.6751 - val_loss: 1.1338 - val_acc: 0.6620
Epoch 3/301
3456/3456 [==============================] - 0s 79us/step - loss: 0.9515 - acc: 0.7237 - val_loss: 0.9035 - val_acc: 0.7350
Epoch 4/301
3456/3456 [==============================] - 0s 87us/step - loss: 0.7930 - acc: 0.7584 - val_loss: 0.8182 - val_acc: 0.7604
Epoch 5/301
3456/3456 [==============================] - 0s 78us/step - loss: 0.7169 - acc: 0.7694 - val_loss: 0.7769 - val_acc: 0.7269
Epoch 6/301
3456/3456 [

### LSTM

In [73]:
# Train 
# One hot encoding of names
ORIG_LENGTH = 12
classes = 28
LR = 1e-3
BATCH_SIZE = 32
EPOCHS = 500
namesA = recordlinker.preprocess.create_training_set(iowa_matches, 'lname1915',
                                                     max_length=ORIG_LENGTH, 
                                                     embed_type='letters', 
                                                     normalize=False)

namesB = recordlinker.preprocess.create_training_set(iowa_matches, 'lname1940',
                                                     max_length=ORIG_LENGTH, 
                                                     embed_type='letters',
                                                     normalize=False)

namesA_onehot = to_categorical(namesA, classes) 
namesB_onehot = to_categorical(namesB, classes)

for latent_dim in LATENT_DIM: 
    save_path = '/Users/kailinlu/Desktop/QMSSWork/RecordLinking/models/lstm_letter_{}_iowa_last/'.format(latent_dim)
    run_id = 'lstm_{}'.format(latent_dim)
    lstm_vae = recordlinker.model.LSTMVAE(batch_size=BATCH_SIZE, 
                                          timesteps=ORIG_LENGTH, 
                                          orig_dim=classes,
                                          latent_dim=latent_dim,
                                          encode_dim=64, 
                                          decode_dim=64,
                                          lr=LR) 
    model_lstm, model_encoder, model_decoder = lstm_vae.train(namesA_onehot, namesB_onehot, 
                                                              epochs=EPOCHS, 
                                                              run_id=run_id, 
                                                              save_path=save_path, 
                                                              earlystop=False,
                                                              tensorboard=True, 
                                                              reconstruct=True, 
                                                              reconstruct_display=10)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12, 28)       0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 12, 64)       23808       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 12, 64)       33024       lstm_1[0][0]                     
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 768)          0           lstm_2[0][0]                     
__________________________________________________________________________________________________
mu (Dense)

3456/3456 [==============================] - 3s 1ms/step - loss: 10.7071 - acc: 0.7329 - val_loss: 11.7249 - val_acc: 0.7189
Epoch 33/500
3456/3456 [==============================] - 4s 1ms/step - loss: 10.5194 - acc: 0.7397 - val_loss: 11.4005 - val_acc: 0.7269
Epoch 34/500
3456/3456 [==============================] - 3s 1ms/step - loss: 10.3014 - acc: 0.7449 - val_loss: 11.4332 - val_acc: 0.7268
Epoch 35/500
3456/3456 [==============================] - 3s 1ms/step - loss: 10.1641 - acc: 0.7487 - val_loss: 11.4507 - val_acc: 0.7231
Epoch 36/500
3456/3456 [==============================] - 3s 1ms/step - loss: 9.8460 - acc: 0.7586 - val_loss: 10.9182 - val_acc: 0.7411
Epoch 37/500
3456/3456 [==============================] - 3s 1ms/step - loss: 9.6768 - acc: 0.7642 - val_loss: 10.7868 - val_acc: 0.7439
Epoch 38/500
3456/3456 [==============================] - 3s 1ms/step - loss: 9.4155 - acc: 0.7717 - val_loss: 10.7158 - val_acc: 0.7484
Epoch 39/500
3456/3456 [==========================

3456/3456 [==============================] - 3s 1ms/step - loss: 4.1658 - acc: 0.9089 - val_loss: 8.0094 - val_acc: 0.8464
Epoch 85/500
3456/3456 [==============================] - 3s 999us/step - loss: 4.0934 - acc: 0.9107 - val_loss: 8.1491 - val_acc: 0.8428
Epoch 86/500
3456/3456 [==============================] - 4s 1ms/step - loss: 4.0460 - acc: 0.9113 - val_loss: 8.0579 - val_acc: 0.8488
Epoch 87/500
3456/3456 [==============================] - 3s 1ms/step - loss: 4.0504 - acc: 0.9121 - val_loss: 8.0348 - val_acc: 0.8503
Epoch 88/500
3456/3456 [==============================] - 3s 1ms/step - loss: 4.0743 - acc: 0.9101 - val_loss: 8.1311 - val_acc: 0.8478
Epoch 89/500
3456/3456 [==============================] - 4s 1ms/step - loss: 3.9062 - acc: 0.9154 - val_loss: 7.9876 - val_acc: 0.8536
Epoch 90/500
3456/3456 [==============================] - 3s 997us/step - loss: 3.9703 - acc: 0.9146 - val_loss: 8.1903 - val_acc: 0.8469
Epoch 91/500
3456/3456 [==============================] -

Epoch 136/500
3456/3456 [==============================] - 3s 993us/step - loss: 2.1444 - acc: 0.9564 - val_loss: 8.4280 - val_acc: 0.8709
Epoch 137/500
3456/3456 [==============================] - 3s 1ms/step - loss: 2.0643 - acc: 0.9583 - val_loss: 8.4119 - val_acc: 0.8709
Epoch 138/500
3456/3456 [==============================] - 3s 1ms/step - loss: 2.1583 - acc: 0.9551 - val_loss: 8.5045 - val_acc: 0.8655
Epoch 139/500
3456/3456 [==============================] - 3s 993us/step - loss: 2.3410 - acc: 0.9503 - val_loss: 8.4562 - val_acc: 0.8667
Epoch 140/500
3456/3456 [==============================] - 3s 1ms/step - loss: 2.2797 - acc: 0.9521 - val_loss: 8.6009 - val_acc: 0.8652
Epoch 141/500
3456/3456 [==============================] - 3s 1ms/step - loss: 2.2127 - acc: 0.9532 - val_loss: 8.7710 - val_acc: 0.8653
Sample Reconstructions:
{'Orig': 'motter', 'Pred:': 'motter'}
{'Orig': 'stover', 'Pred:': 'stover'}
{'Orig': 'bell', 'Pred:': 'bell'}
{'Orig': 'hinsch', 'Pred:': 'hinsch'}
{'

Epoch 188/500
3456/3456 [==============================] - 4s 1ms/step - loss: 1.3911 - acc: 0.9708 - val_loss: 9.2984 - val_acc: 0.8733
Epoch 189/500
3456/3456 [==============================] - 4s 1ms/step - loss: 1.3382 - acc: 0.9715 - val_loss: 9.4139 - val_acc: 0.8762
Epoch 190/500
3456/3456 [==============================] - 4s 1ms/step - loss: 1.4058 - acc: 0.9704 - val_loss: 9.5125 - val_acc: 0.8723
Epoch 191/500
3456/3456 [==============================] - 4s 1ms/step - loss: 1.4937 - acc: 0.9677 - val_loss: 9.5173 - val_acc: 0.8736
Sample Reconstructions:
{'Orig': 'lantry', 'Pred:': 'lantry'}
{'Orig': 'dan', 'Pred:': 'dean'}
{'Orig': 'tuley', 'Pred:': 'tuley'}
{'Orig': 'covey', 'Pred:': 'covey'}
{'Orig': 'lucas', 'Pred:': 'lucas'}
Epoch 192/500
3456/3456 [==============================] - 4s 1ms/step - loss: 1.6271 - acc: 0.9643 - val_loss: 9.6026 - val_acc: 0.8664
Epoch 193/500
3456/3456 [==============================] - 4s 1ms/step - loss: 1.6121 - acc: 0.9652 - val_loss: 

...

3456/3456 [==============================] - 3s 991us/step - loss: 0.9153 - acc: 0.9765 - val_loss: 9.0745 - val_acc: 0.8874
Epoch 281/500
3456/3456 [==============================] - 4s 1ms/step - loss: 1.0525 - acc: 0.9732 - val_loss: 9.1843 - val_acc: 0.8828
Sample Reconstructions:
{'Orig': 'duenow', 'Pred:': 'dunnow'}
{'Orig': 'oherron', 'Pred:': 'oherron'}
{'Orig': 'oehlerts', 'Pred:': 'ochlarts'}
{'Orig': 'pettit', 'Pred:': 'pettit'}
{'Orig': 'rupiper', 'Pred:': 'rupiper'}
Epoch 282/500
3456/3456 [==============================] - 3s 997us/step - loss: 1.3200 - acc: 0.9681 - val_loss: 9.3603 - val_acc: 0.8797
Epoch 283/500
3456/3456 [==============================] - 4s 1ms/step - loss: 1.3656 - acc: 0.9670 - val_loss: 8.8772 - val_acc: 0.8866
Epoch 284/500
3456/3456 [==============================] - 4s 1ms/step - loss: 1.1571 - acc: 0.9730 - val_loss: 9.1461 - val_acc: 0.8871
Epoch 285/500
3456/3456 [==============================] - 4s 1ms/step - loss: 1.1591 - acc: 0.9722 - v

3456/3456 [==============================] - 3s 997us/step - loss: 0.7987 - acc: 0.9785 - val_loss: 9.1433 - val_acc: 0.8913
Sample Reconstructions:
{'Orig': 'lowder', 'Pred:': 'lowder'}
{'Orig': 'stepanek', 'Pred:': 'stepaneh'}
{'Orig': 'counsell', 'Pred:': 'counsell'}
{'Orig': 'sook', 'Pred:': 'sook'}
{'Orig': 'strickler', 'Pred:': 'strickler'}
Epoch 332/500
3456/3456 [==============================] - 3s 989us/step - loss: 0.7951 - acc: 0.9781 - val_loss: 9.2395 - val_acc: 0.8913
Epoch 333/500
3456/3456 [==============================] - 3s 997us/step - loss: 0.7939 - acc: 0.9778 - val_loss: 9.2662 - val_acc: 0.8900
Epoch 334/500
3456/3456 [==============================] - 3s 995us/step - loss: 0.7999 - acc: 0.9780 - val_loss: 9.3099 - val_acc: 0.8905
Epoch 335/500
3456/3456 [==============================] - 4s 1ms/step - loss: 0.8086 - acc: 0.9778 - val_loss: 9.2597 - val_acc: 0.8902
Epoch 336/500
3456/3456 [==============================] - 4s 1ms/step - loss: 0.8220 - acc: 0.97

3456/3456 [==============================] - 3s 999us/step - loss: 1.7682 - acc: 0.9572 - val_loss: 9.9065 - val_acc: 0.8783
Epoch 383/500
3456/3456 [==============================] - 4s 1ms/step - loss: 1.1906 - acc: 0.9710 - val_loss: 9.3768 - val_acc: 0.8879
Epoch 384/500
3456/3456 [==============================] - 4s 1ms/step - loss: 0.9234 - acc: 0.9758 - val_loss: 9.4339 - val_acc: 0.8877
Epoch 385/500
3456/3456 [==============================] - 4s 1ms/step - loss: 0.8232 - acc: 0.9776 - val_loss: 9.5295 - val_acc: 0.8899
Epoch 386/500
3456/3456 [==============================] - 4s 1ms/step - loss: 0.7890 - acc: 0.9779 - val_loss: 9.4332 - val_acc: 0.8884
Epoch 387/500
3456/3456 [==============================] - 4s 1ms/step - loss: 0.7796 - acc: 0.9777 - val_loss: 9.4423 - val_acc: 0.8879
Epoch 388/500
3456/3456 [==============================] - 4s 1ms/step - loss: 0.7717 - acc: 0.9781 - val_loss: 9.4138 - val_acc: 0.8903
Epoch 389/500
3456/3456 [============================

3456/3456 [==============================] - 4s 1ms/step - loss: 1.1706 - acc: 0.9695 - val_loss: 9.7825 - val_acc: 0.8831
Epoch 434/500
3456/3456 [==============================] - 4s 1ms/step - loss: 1.2690 - acc: 0.9681 - val_loss: 10.0033 - val_acc: 0.8850
Epoch 435/500
3456/3456 [==============================] - 3s 1ms/step - loss: 1.0065 - acc: 0.9743 - val_loss: 9.5333 - val_acc: 0.8904
Epoch 436/500
3456/3456 [==============================] - 4s 1ms/step - loss: 0.8788 - acc: 0.9770 - val_loss: 9.5584 - val_acc: 0.8888
Epoch 437/500
3456/3456 [==============================] - 4s 1ms/step - loss: 0.7820 - acc: 0.9785 - val_loss: 9.5924 - val_acc: 0.8899
Epoch 438/500
3456/3456 [==============================] - 3s 1ms/step - loss: 0.7570 - acc: 0.9776 - val_loss: 9.5373 - val_acc: 0.8890
Epoch 439/500
3456/3456 [==============================] - 3s 1ms/step - loss: 0.7349 - acc: 0.9786 - val_loss: 9.6871 - val_acc: 0.8900
Epoch 440/500
3456/3456 [=============================

Epoch 485/500
3456/3456 [==============================] - 3s 986us/step - loss: 0.7813 - acc: 0.9775 - val_loss: 9.7483 - val_acc: 0.8895
Epoch 486/500
3456/3456 [==============================] - 3s 976us/step - loss: 0.8531 - acc: 0.9761 - val_loss: 10.0664 - val_acc: 0.8832
Epoch 487/500
3456/3456 [==============================] - 3s 977us/step - loss: 1.2402 - acc: 0.9675 - val_loss: 9.9694 - val_acc: 0.8807
Epoch 488/500
3456/3456 [==============================] - 3s 1ms/step - loss: 1.2363 - acc: 0.9678 - val_loss: 9.7814 - val_acc: 0.8829
Epoch 489/500
3456/3456 [==============================] - 3s 992us/step - loss: 1.0700 - acc: 0.9726 - val_loss: 9.7407 - val_acc: 0.8859
Epoch 490/500
3456/3456 [==============================] - 3s 989us/step - loss: 0.9945 - acc: 0.9734 - val_loss: 9.8509 - val_acc: 0.8886
Epoch 491/500
3456/3456 [==============================] - 3s 989us/step - loss: 0.8097 - acc: 0.9774 - val_loss: 9.6915 - val_acc: 0.8878
Sample Reconstructions:
{'Or